In [ ]:
from glob import glob
import os
from skimage import io
import pandas as pd
import numpy as np
import napari
from matplotlib import pyplot as plt
import seaborn as sns
from skimage.measure import find_contours

from stardist.models import StarDist2D
from csbdeep.utils import Path, normalize
from skimage.segmentation import find_boundaries, expand_labels
from skimage.measure import regionprops_table
from skimage.morphology import label, disk
from scipy.ndimage import binary_dilation, binary_erosion
from sklearn.neighbors import NearestNeighbors
import nd2
import napari
os.chdir('E:/Dropbox (VU Basic Sciences)')

In [ ]:
model = StarDist2D(None, name='Mary_Nuc_2D_20X', basedir='Python Scripts/From ubuntu/models')
axis_norm = (0,1)
n_channel = 1

In [ ]:
def parse_filename(image):
    _, celltype, ND, chan1, chan2, _ , field, _= os.path.basename(image).split('.')
    plate = ND[3:]
    ND = ND[:3]

    chan1 = chan1.split('-')[0]
    chan2 = chan2.split('-')[0]
    return celltype, ND, plate, chan1, chan2, field


In [ ]:
def dilate_erode_mask(image):
    # Subtract test_image mean from test_image
    corrected_image = image - np.mean(image)
    corrected_image = np.clip(corrected_image, 0, None)

    percentile = np.percentile(corrected_image, 95)
    test_mask = (corrected_image > percentile*0.55).astype(int)
    pre_dilated_mask = binary_dilation(test_mask, structure=disk(1))
    eroded_mask = binary_erosion(pre_dilated_mask, structure=disk(6))
    dilated_mask = binary_dilation(eroded_mask, structure=disk(3))
    test_image_dilated = np.where(dilated_mask, corrected_image, 0)
    return test_mask, corrected_image, test_image_dilated, dilated_mask

In [ ]:
l_images = glob('2023-12-22 CRs ND16 neuron markers/tuj1 pax6/*.nd2')


In [ ]:
fig, axes = plt.subplots(len(l_images), 2, figsize=(20, 10 * len(l_images)))

for i, image in enumerate(l_images):
    test_array = nd2.imread(image)
    if len(test_array.shape) == 3:
        continue
    max_array = np.max(test_array, axis = 0)
    img = normalize(max_array[-1], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)
    axes[i, 0].imshow(max_array[2], cmap = 'gray')
    axes[i, 1].imshow(labels)
    axes[i, 1].set_title(os.path.basename(image))

plt.savefig('2024-03-08 Mary_Combined_Re-Thresholding/2d_crs_tuj1_segmentation.png')

In [ ]:
viewer = napari.Viewer()
test_array = nd2.imread(l_images[0])
max_array = np.max(test_array, axis = 0)

img = normalize(max_array[-1], 1,99.8, axis=axis_norm)
labels, details = model.predict_instances(img)

viewer.add_image(max_array[-1], name='DAPI')
viewer.add_image(max_array[0], name='Pax6')
viewer.add_image(max_array[1], name='Tuj1')
viewer.add_labels(labels, name='Segmentation')




In [ ]:
df_all = pd.DataFrame()
for field in l_images:
    image = nd2.imread(field)
    dapi = np.max(image, axis = 0)[-1]
    tuj1 = np.max(image, axis = 0)[1]
    pax6 = np.max(image, axis = 0)[0]

    # segment on dapi
    img = normalize(dapi, 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)
    exp_lab = expand_labels(labels, distance = 10)

    tuj1_props = regionprops_table(exp_lab, intensity_image = tuj1, properties = ['label', 'mean_intensity', 'centroid'])
    df_tuj1 = pd.DataFrame(tuj1_props)
    df_tuj1.rename(columns = {'mean_intensity': 'tuj1'}, inplace = True)

    pax6_props = regionprops_table(labels, intensity_image = pax6, properties = ['label', 'mean_intensity'])
    df_pax6 = pd.DataFrame(pax6_props)
    df_pax6.rename(columns = {'mean_intensity': 'pax6'}, inplace = True)
    df_combined = df_tuj1.merge(df_pax6, on = 'label')

    test_mask, corrected_tuj1, tuj1_dilated, dilated_mask = dilate_erode_mask(tuj1)
    labeled_mask = label(dilated_mask)
    label_props = pd.DataFrame(regionprops_table(labeled_mask, intensity_image = tuj1, properties = ['label', 'centroid', 'area', 'mean_intensity']))


    nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(df_combined[['centroid-0', 'centroid-1']])

    for point in label_props.iterrows():
        if point[1]['area'] < 200:
            continue
        if point[1]['mean_intensity'] < 0:
            continue
        point_x = np.array([point[1]['centroid-0'], point[1]['centroid-1']])
        distances, indices = nbrs.kneighbors([point_x])
        df_combined.loc[indices[0][0], 'Tuj1+ Matched'] = True
        df_combined.loc[indices[0][0], 'Tuj1+ Matched Area'] = point[1]['area']
        df_combined.loc[indices[0][0], 'Tuj1+ Matched Intensity'] = point[1]['mean_intensity']
        
    df_combined['FileName'] = os.path.basename(field)
    celltype, ND, plate, chan1, chan2, field = parse_filename(os.path.basename(field))
    df_combined['CellType'] = celltype
    df_combined['ND'] = ND
    df_combined['Plate'] = plate
    df_combined['field'] = field

    df_all = df_all.append(df_combined, ignore_index = True)

In [ ]:
fig, axes = plt.subplots(len(l_images), 4, figsize=(20, 100))

for i, field in enumerate(l_images):
    tuj1 = np.max(nd2.imread(field), axis=0)[1]
    test_mask, corrected_tuj1, tuj1_dilated, dilated_mask = dilate_erode_mask(tuj1)
    io.imshow(tuj1, ax = axes[i, 0],cmap = 'gray', vmax = 400)
    axes[i, 0].set_title(os.path.basename(field))
    #io.imshow(corrected_tuj1, ax = axes[i, 1], vmin = 0)
    #io.imshow(dilated_mask, ax = axes[i, 2])
    #io.imshow(tuj1, cmap = 'gray', ax = axes[i, 3], vmin = 0)
    axes[i, 2].imshow(tuj1, cmap = plt.cm.gray, vmax = 400)
    contours = find_contours(dilated_mask)
    for contour in contours:
        axes[i,2].plot(contour[:, 1], contour[:, 0], linewidth=1)
    io.imshow(test_mask, ax = axes[i, 1])

    dapi = np.max(nd2.imread(field), axis=0)[-1]

    img = normalize(dapi, 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)
    exp_lab = expand_labels(labels, distance = 10)

    tuj1_props = regionprops_table(exp_lab, intensity_image = tuj1, properties = ['label', 'centroid'])
    df_tuj1 = pd.DataFrame(tuj1_props)

    labeled_mask = label(dilated_mask)
    label_props = pd.DataFrame(regionprops_table(labeled_mask, intensity_image = tuj1, properties = ['label', 'centroid', 'area', 'mean_intensity']))
    
    corrected_image = tuj1 - np.mean(tuj1)
    corrected_image = np.clip(corrected_image, 0, None)
    percentile = np.percentile(corrected_image, 97)
    
    
    nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(df_tuj1[['centroid-0', 'centroid-1']])
    for point in label_props.iterrows():
        if point[1]['area'] < 75:
            continue
        if point[1]['mean_intensity'] < percentile*0.15:
            continue
        point_x = np.array([point[1]['centroid-0'], point[1]['centroid-1']])
        distances, indices = nbrs.kneighbors([point_x])
        df_tuj1.loc[indices[0][0], 'Tuj1+ Matched'] = True
        df_tuj1.loc[indices[0][0], 'Tuj1+ Matched Area'] = point[1]['area']
        df_tuj1.loc[indices[0][0], 'Tuj1+ Matched Intensity'] = point[1]['mean_intensity']
    
    axes[i, 3].imshow(tuj1, cmap = plt.cm.gray, vmax = 400)  
    if df_tuj1.shape[1] > 3:
        df_filtered = df_tuj1[df_tuj1['Tuj1+ Matched'] == True]
        axes[i, 3].scatter(df_filtered['centroid-1'], df_filtered['centroid-0'], c='r', s=1)


plt.tight_layout()

plt.savefig('2024-03-08 Mary_Combined_Re-Thresholding/2d_crs_tuj1_thresholding.png')

In [ ]:
for index, x in enumerate(l_images):
    print('index = ' , index, 'file = ', os.path.basename(x))

test_index = 5
df_display = df_all[df_all['FileName'] == os.path.basename(l_images[test_index])]
coords = df_display[['centroid-0', 'centroid-1']].values

tuj1_pos = df_display[df_display['Tuj1+ Matched'] == True]
tuj1_coords = tuj1_pos[['centroid-0', 'centroid-1']]

test_mask, corrected_tuj1, tuj1_dilated, dilated_mask = dilate_erode_mask(np.max(nd2.imread(l_images[test_index]), axis = 0)[2])
labeled_mask = label(dilated_mask)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(np.max(nd2.imread(l_images[test_index]), axis = 0)[-1], name='DAPI')
viewer.add_image(np.max(nd2.imread(l_images[test_index]), axis = 0)[0], name='Pax6')
viewer.add_image(np.max(nd2.imread(l_images[test_index]), axis = 0)[1], name='Tuj1')

viewer.add_points(coords, properties = df_display[['tuj1', 'pax6']].round(2), size = 5, face_color = 'red')
viewer.add_points(tuj1_coords, properties = tuj1_pos[['tuj1', 'Tuj1+ Matched Area', 'Tuj1+ Matched Intensity']].round(2), size = 5, face_color = 'blue')

In [ ]:

df_all['Pax6+'] = df_all['pax6'] > 120
df_all['double_pos'] = (df_all['Tuj1+ Matched'] == True) & (df_all['Pax6+'] == True)

grouped_df = df_all.groupby(['FileName'])
result_export = []
for key, item in grouped_df:
    a_group = grouped_df.get_group(key)
    celltype = a_group['CellType'].values[0]
    ND = a_group['ND'].values[0]
    plate = a_group['Plate'].values[0]
    field = a_group['field'].values[0]
    
    tot_cell = len(a_group.index)

    pos_chan1 = a_group['Tuj1+ Matched'].count()
    pos_chan2 = a_group['Pax6+'].sum()
    double_pos = a_group['double_pos'].sum()
    single_export = [key, celltype, ND, field, plate, tot_cell, pos_chan1, pos_chan1/tot_cell * 100, pos_chan2, pos_chan2/tot_cell * 100, double_pos, double_pos/pos_chan1 * 100]

    result_export.append(single_export)

In [ ]:
df_export = pd.DataFrame(result_export, columns = ['FileName', 'CellType', 'ND', 'Field', 'Plate', 'Total_Cell','Tuj1+' ,'Tuj1 Percent Positive', 'Pax6+', 'Pax6 Percent Positive', 'Double Positive', 'Double Positive Percent'])


In [ ]:
df_export.to_csv('2024-03-08 Mary_Combined_Re-Thresholding/2d_crs_perfile_stats.csv', index = False)

In [ ]:
sns.barplot(data = df_export, x = 'Plate', y = 'Tuj1 Percent Positive', hue = 'CellType')

In [ ]:
sns.barplot(data = df_export, x = 'Plate', y = 'Double Positive Percent', hue = 'CellType')